todo list: https://trello.com/b/APhmhWw4/robot-vision-project

sources:
https://answers.opencv.org/question/144252/perspective-transform-without-crop/
https://stackoverflow.com/questions/6087241/opencv-warpperspective
https://stackoverflow.com/questions/44457064/displaying-stitched-images-together-without-cutoff-using-warpaffine/44459869#44459869

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from utils import get_four_points
%matplotlib inline

look into warpPerspective arguments/parameters 
- there is no argument that will allow me to get the whole warped image to show 
- what i can do is calculate the homography on the four corners of the original image (the positions) and see where they end up, then all i need to do is translate that warped image so that all of it lies in the 1st quadrant of the coordinate plane 

background subtraction method

ways to get a more accurate warping result:
april tags/four edges/pairwise intersection 
take a look at the vanishing points for the crosswalk markings

add tracking of moving objects

object detection
bounding box, center point as position on ground

cornernet with pytorch - more accurate object detection 


In [ ]:
# getting homography matrix
cap = cv2.VideoCapture('./video_data/columbus_at_wash1.mkv')

if not cap.isOpened():
    print("Error opening")
else:
    while True:
        ret, frame = cap.read()
        if not ret: 
            break
            
        cv2.imshow("img", frame)
        cv2.waitKey(0)

cap.release()
cv2.destroyAllWindows()

In [3]:
# getting homography matrix
cap = cv2.VideoCapture('./video_data/columbus_at_wash1.mkv')

if not cap.isOpened():
    print("Error opening")
else:
    ret, frame = cap.read()
    size = (20,100,3)
    
    # points the source will get mapped to 
    pts_dst = np.array([
        [800,800],
        [800+size[0]-1,800],
        [800+size[0]-1,800+size[1]-1],
        [800,800+size[1]-1]],dtype=float)
    
    
    # need to start from top left and end with bottom left
    pts_src = get_four_points(frame)

In [13]:
cap = cv2.VideoCapture('./video_data/columbus_at_wash1.mkv')
if not cap.isOpened():
    print("Error opening")
else:
    ret, frame = cap.read()
    
    # height/width to get the coordinates of four corners of original image
    height, width = frame.shape[:2]
    topleft = [0,0,1]
    topright = [0,width,1]
    bottomleft = [height,0,1]
    bottomright = [height,width,1]
    
    h, status = cv2.findHomography(pts_src, pts_dst)
    
    # find where the four corners lie after homography 
    topleftH = np.dot(topleft,h)
    toprightH = np.dot(topright,h)
    botleftH = np.dot(bottomleft,h)
    botrightH = np.dot(bottomright,h)
    
    # figure out the minimum values
    x_coord = np.array([topleftH[0],toprightH[0],botleftH[0],botrightH[0]])
    y_coord = np.array([topleftH[1],toprightH[1],botleftH[1],botrightH[1]])
    x_min = np.min(x_coord)
    y_min = np.min(y_coord)
    x_max = np.max(x_coord)
    y_max = np.max(y_coord)
    print(x_min)
    print(y_min)
    print(x_max)
    print(y_max)
    
    # calculate warped image size using min/max values
#     newsize = (np.ceil(x_max - x_min).astype(int)//8, 
#                np.ceil(y_max - y_min).astype(int)//8, 3)
    newsize = (4000,6000,3)
    
    im_dst = np.zeros(newsize, np.uint8)
    
    # use the minimums to create translation matrix
#     translate = np.array([
#         [1,0,-x_min],
#         [0,1,-y_min],
#         [0,0,1]
#     ])
    
    translate = np.array([
        [1,0,100],
        [0,1,3000],
        [0,0,1]
    ])
    
    # multiply homography and translation to get total transformation
    transform = np.dot(translate,h)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
#             im_dst = cv2.warpPerspective(frame, h, newsize[0:2])
            im_dst = cv2.warpPerspective(frame, transform, newsize[0:2])
            height, width = im_dst.shape[:2]
            newimage = cv2.resize(im_dst, (width//4, height//4))
            cv2.imshow("image", newimage)
#             cv2.imshow("image", im_dst)
            if cv2.waitKey(1) == ord('q'):
                break 
    
    print(newsize)
    print(im_dst.shape)
    
cap.release()
cv2.destroyAllWindows()

-2150.5977593644548
-12170.668140497146
-0.0001567661718623023
-0.004065025656033303
(4000, 6000, 3)
(6000, 4000, 3)
